<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/distilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def softmax_sparse_categorical_crossentropy(labels, logits):
  softmaxed = tf.keras.backend.softmax(logits)

  return tf.keras.losses.sparse_categorical_crossentropy(labels, softmaxed)

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [4]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [5]:
ds_info.features['image'].shape

(28, 28, 1)

In [6]:
num_training_examples = ds_info.splits['train'].num_examples

In [125]:
ds_train_iter = ds_train.as_numpy_iterator()
train_images, train_labels = zip(*[ds_train_iter.next() for i in range(num_training_examples)])

In [155]:
num_testing_examples = ds_info.splits['test'].num_examples

In [156]:
ds_test_iter = ds_test.as_numpy_iterator()
test_images, test_labels = zip(*[ds_test_iter.next() for i in range(num_testing_examples)])

In [126]:
train_images = np.asarray(train_images)

In [158]:
test_images = np.asarray(test_images)

In [127]:
train_images.shape

(60000, 28, 28, 1)

In [128]:
# ds_train = ds_train.take(10)

In [129]:
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [148]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [131]:
control_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#control_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
control_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
control_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_11 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_11 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_12 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_12 (Dropout)         (None, 6, 6, 64)         

In [139]:
control_model.fit(ds_train, epochs=30)

Epoch 1/30
469/469 [==============================] - 2s 4ms/step - loss: 0.9866 - accuracy: 0.7298
Epoch 2/30
469/469 [==============================] - 2s 4ms/step - loss: 0.3935 - accuracy: 0.9004
Epoch 3/30
469/469 [==============================] - 2s 4ms/step - loss: 0.2721 - accuracy: 0.9270
Epoch 4/30
469/469 [==============================] - 2s 4ms/step - loss: 0.2200 - accuracy: 0.9382
Epoch 5/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1960 - accuracy: 0.9435
Epoch 6/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1773 - accuracy: 0.9476
Epoch 7/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1685 - accuracy: 0.9494
Epoch 8/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1549 - accuracy: 0.9529
Epoch 9/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1468 - accuracy: 0.9553
Epoch 10/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1428 - accuracy: 0.9561

In [143]:
test_loss, test_acc = control_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 2s 21ms/step - loss: 0.0732 - accuracy: 0.9778
Test Loss: 0.07322971522808075
Test Accuracy: 0.9778000116348267


In [26]:
teacher_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#teacher_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
teacher_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
teacher_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 6, 64)         

In [140]:
teacher_model.fit(ds_train, epochs=30)

Epoch 1/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0547 - accuracy: 0.9825
Epoch 2/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0511 - accuracy: 0.9842
Epoch 3/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0469 - accuracy: 0.9854
Epoch 4/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0452 - accuracy: 0.9857
Epoch 5/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0415 - accuracy: 0.9870
Epoch 6/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0384 - accuracy: 0.9877
Epoch 7/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0373 - accuracy: 0.9882
Epoch 8/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0350 - accuracy: 0.9887
Epoch 9/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0327 - accuracy: 0.9894
Epoch 10/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0305 - accuracy: 0.9900

In [28]:
test_loss, test_acc = teacher_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 2s 20ms/step - loss: 0.0383 - accuracy: 0.9886
Test Loss: 0.03831847384572029
Test Accuracy: 0.9886000156402588


In [111]:
train_images = list(map(lambda x:tf.reshape(x,(-1,28,28,1)), train_images))

In [132]:
ds_train_images = tf.data.Dataset.from_tensor_slices(list(map(lambda x:tf.reshape(x,(-1,28,28,1)), train_images)))

In [133]:
ds_train_images

<TensorSliceDataset shapes: (1, 28, 28, 1), types: tf.float32>

In [141]:
soft_labels = teacher_model.predict(ds_train_images,verbose=1)

60000/60000 [==============================] - 80s 1ms/step


In [135]:
temperature = 3
afterwards_temperature = 1

def temperature_softmax(logits):
  soft_logits = tf.keras.backend.exp(logits / temperature)
  return soft_logits / tf.keras.backend.sum(soft_logits, axis=-1, keepdims=True) / afterwards_temperature

def distillation_loss(labels, logits):
  labels = temperature_softmax(labels)
  logits = temperature_softmax(logits)

  return -tf.keras.backend.mean(labels * tf.keras.backend.log(logits))

In [136]:
student_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  # tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

student_model.compile('adam', distillation_loss, ['accuracy'])
student_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_13 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_13 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_14 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_14 (Dropout)         (None, 6, 6, 64)         

In [142]:
student_model.fit(train_images, soft_labels, epochs=30)

Epoch 1/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0395 - accuracy: 0.9525
Epoch 2/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0373 - accuracy: 0.9581
Epoch 3/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0371 - accuracy: 0.9585
Epoch 4/30
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0370 - accuracy: 0.9590
Epoch 5/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0368 - accuracy: 0.9605
Epoch 6/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0368 - accuracy: 0.9603
Epoch 7/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0367 - accuracy: 0.9605
Epoch 8/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0366 - accuracy: 0.9615
Epoch 9/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0365 - accuracy: 0.9625
Epoch 10/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.036

In [162]:
test_images.shape, test_labels.shape

AttributeError: ignored

In [164]:
test_loss, test_acc = student_model.evaluate(ds_train_images)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

    1/60000 [..............................] - ETA: 22s - loss: 0.0000e+00 - accuracy: 0.0000e+00WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0008s vs `on_test_batch_end` time: 0.0038s). Check your callbacks.


 9255/60000 [===>..........................] - ETA: 1:01 - loss: 0.0000e+00 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored